<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/object_detection_for_image_tagging%20/flower_fruit/flower_fruit_generate_tags_yolov3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using YOLO v3 pre-trained on Google Open Images to add flower/fruit tags to plant images
---
*Last Updated 8 June 2023*   
Using a YOLOv3 model (downloaded from [here](https://github.com/AlexeyAB/darknet) ) pre-trained on [Google Open Images](https://storage.googleapis.com/openimages/web/visualizer/index.html?set=train&type=detection&c=%2Fm%2F03vt0) as a method to do customized, large-scale image processing. EOL Angiosperm images will be tagged for flower/fruit present using object detection. Tags will further extend EOLv3 image search functions.

Notes:   
* Run code blocks by pressing play button in brackets on left
* Change parameters using form fields on right (find details at corresponding lines of code by searching '#@param')

References:   
* Check out [AlexeyAB's darknet repo](https://github.com/AlexeyAB/darknet) for Colab tutorials like [this one](https://colab.research.google.com/drive/12QusaaRj_lUwCGDvQNfICpa7kA7_a2dE).

## Installs & Imports
---

In [ ]:
#@title Choose where to save results
import os

# Use dropdown menu on right
save = "in Colab runtime (files deleted after each session)" #@param ["in my Google Drive", "in Colab runtime (files deleted after each session)"]

# Mount google drive to export image tagging file(s)
if 'Google Drive' in save:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

# Type in the path to your project wd in form field on right
basewd = "/content/drive/MyDrive/train" #@param ["/content/drive/MyDrive/train"] {allow-input: true}

# Type in the folder that you want to contain TF2 files
folder = "darknet" #@param ["darknet"] {allow-input: true}
cwd = basewd + '/' + folder

# Download helper_funcs folder
!pip3 -q install --upgrade gdown
!gdown 1-BaybePbv810CTuHrF8JGRJfuEsHRQYH
!tar -xzvf helper_funcs.tar.gz -C .

# Install requirements.txt
!pip3 -q install -r requirements.txt
print('\n\n\n \033[91m If ERROR from pip dependency solver listed above, restart runtime then run this cell again to refresh installed package versions.')

In [ ]:
#@title Set up directory structure & make darknet
!python setup.py $cwd $basewd

In [ ]:
#@title Import libraries

# For importing/exporting files, working with arrays, etc
import glob
import pathlib
import six.moves.urllib as urllib
import sys
import tarfile
import zipfile
import numpy as np 
import csv
import time
import pandas as pd

# For downloading images
!apt-get -qq install aria2

# For drawing onto and plotting images
import matplotlib.pyplot as plt
import cv2
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
sys.path.append('/content')

### Prepare object detection functions and settings

In [ ]:
#@title Define functions
from wrangle_data import read_datafile, imShow

# Display full URLs in outputs so you can click them and inspect images
pd.set_option('display.max_colwidth', None)

# Define start and stop indices in EOL bundle for running inference   
def set_start_stop(df):
    # To test with a tiny subset, use 5 random bundle images
    if "tiny subset" in run:
        start=np.random.choice(a=1000, size=1)[0]
        stop=start+5
    # To run inference on 4 batches of 5k images each
    elif "_a." in outfpath: # batch a is from 0-5000
        start=0
        stop=5000
    elif "_b." in outfpath: # batch b is from 5000-1000
        start=5000
        stop=10000
    elif "_c." in outfpath: # batch c is from 10000-15000
        start=10000
        stop=15000
    elif "_d." in outfpath: # batch d is from 15000-20000
        start=15000
        stop=20000
    
    return start, stop

# For uploading an image from url
# Modified from https://www.pyimagesearch.com/2015/03/02/convert-url-to-image-with-python-and-opencv/
def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    im_h, im_w = image.shape[:2]
 
    return image

# Combine individual prediction files for each image to all_predictions.txt
def combine_predictions(imgs_dir):
    # Delete inference images file list
    !rm $outfpath
    # Combine inference text files for each image and save to all_predictions.txt
    fns = os.listdir(imgs_dir)
    with open('data/results/all_predictions.txt', 'w') as outfile:
        header = "class_id x y w h img_id"
        outfile.write(header + "\n")
        for fn in fns:
            if '.txt' in fn:
                with open('data/imgs/'+fn) as infile:
                    lines = infile.readlines()
                    newlines = [''.join([x.strip(), ' ' + os.path.splitext(fn)[0] + '\n']) for x in lines]
                    outfile.writelines(newlines)
    # Load all_predictions.txt
    df = pd.read_csv('data/results/all_predictions.txt')
    print("Model predictions: \n", df.head())

    return df

# Combine tagging files for batches A-D
def combine_tag_files(tags_fpath):
    # Combine tag files for batches A-D
    fpath =  os.path.splitext(tags_fpath)[0]
    base = fpath.rsplit('_',1)[0] + '_'
    exts = ['a.tsv', 'b.tsv', 'c.tsv', 'd.tsv'] 
    all_filenames = [base + e for e in exts]
    df = pd.concat([pd.read_csv(f, sep='\t', header=0, na_filter = False) for f in all_filenames], ignore_index=True)
    # Choose desired columns for tagging
    df = df[['url', 'img_id', 'class_id']]
    df.rename(columns={'url': 'eolMediaURL', 'img_id': 'identifier', 'class_id': 'tag'}, inplace=True)
    print("\nNew concatenated dataframe with all 4 batches: \n", df[['eolMediaURL', 'tag']].head())

    return df

def add_class_names(all_predictions):
    # Model predictions with number-coded classes
    numbered_tags = pd.read_csv(all_predictions, header=0, sep=" ")
    numbered_tags.class_id = numbered_tags.class_id - 1 # python counts from 0, Yolo from 1
    print("\nModel predictions by class id (number): \n", numbered_tags)

    # Add class names to model predictions
    classes = pd.read_table('data/openimages.names')
    classes.columns = ['name']
    classes_dict = pd.Series(classes.name.values, index=classes.index).to_dict()
    tags = numbered_tags.copy()
    tags.replace({"class_id":classes_dict}, inplace=True)
    tags['class_id'] = tags['class_id'].astype(str)
    print("\nModel predictions by class id (name): \n", tags)

    return tags

# Add EOL media URL's to named image tags
def add_eolMediaURLs(tags, bundle):
    # Read in EOL 20k image url bundle
    bundle = read_datafile(bundle)
    bundle.columns = ['url']
    
    # Map eolMediaURLs to tags using image filenames
    img_fns = bundle['url'].apply(lambda x: os.path.splitext((os.path.basename(x)))[0])
    bundle['img_id'] = img_fns
    # Make datatypes consistent for bundle and tags
    bundle['img_id'] = bundle['img_id'].astype("string")
    tags['img_id'] = tags['img_id'].astype("string")
    # Add URLs to tags with img_id as a key
    final_tags = tags.merge(bundle, on='img_id')
    final_tags.reset_index(drop=True, inplace=True)
    final_tags.drop_duplicates(inplace=True, ignore_index=True)
    print("\nModel predictions with EOL media URL's added: \n", final_tags.head())

    return final_tags

# Set filename for saving classification results
def set_outpath(tags_file):
    tags_file = os.path.splitext(tags_file)[0]
    outpath = cwd + '/data/results/' + tags_file + '.tsv'
    print("Saving results to: \n", outpath)

    return outpath

## Generate tags for images
---
Run EOL 20k image bundles through pre-trained object detection models and save results in 4 batches (A-D). 

#### Test: Run individual image through by filename and display results

In [ ]:
#@title Run with sample EOL Angiosperm image (To test with your own image, upload file to data/imgs and update fn formfield)

# Run with sample EOL image
# Define filepath to image
fn = "542.8209936861.jpg" #@param ["542.8209936861.jpg"] {allow-input: true}
img_fpath = 'data/imgs/' + fn

# Download image
%cd $cwd
%cd data/imgs
!gdown 1A7dHlbnAlRS-pHwS2QHg--HzTQc1CLx3
%cd $cwd

# Run darknet and show bounding box coordinates
!./darknet detector test cfg/openimages.data cfg/yolov3-openimages.cfg yolov3-openimages.weights {img_fpath}

# Display detection results
print("\nObject detection results:")
imShow('predictions.jpg')

### Generate tags: Run inference on EOL images & save results for tagging - Run 4X for batches A-D
Use 20K EOL Angiosperm image bundle to identify present flowers and fruits. Results are saved to [tags_file].tsv. Run this section 4 times (to make batches A-D) of 5K images each to incrementally save in case of Colab timeouts.

In [ ]:
#@title Enter EOL image bundle and choose inference settings. Change **tags_file** for each batch A-D
%cd $cwd

# Load in EOL Angiosperm 20k image bundle
bundle = "https://editors.eol.org/other_files/bundle_images/files/images_for_Angiosperms_20K_breakdown_download_000030.txt" #@param ["https://editors.eol.org/other_files/bundle_images/files/images_for_Angiosperms_20K_breakdown_download_000030.txt"] {allow-input: true}
df = read_datafile(bundle)
print("EOL image bundle with {} images: \n{}".format(len(df), df.head()))

# Test pipeline with a smaller subset than 5k images?
run = "test with tiny subset" #@param ["test with tiny subset", "for all images"]

# Display detection results on images?
display_results = "yes (use this option if testing tiny subsets; only works for \u003C50 images)" #@param ["yes (use this option if testing tiny subsets; only works for \u003C50 images)", "no (use this option if running batches)"]

# Take 5k subset of bundle for running inference
# Change filename for each batch
tags_file = "flower_fruit_tags_d" #@param ["flower_fruit_tags_a", "flower_fruit_tags_b", "flower_fruit_tags_c", "flower_fruit_tags_d"] {allow-input: true}
tags_file = tags_file + ".txt"
imgs_dir = "data/imgs/"
outfpath = imgs_dir + tags_file
print("\nSaving tagging results to: \n{}".format(outfpath))

# Add 5k subset of image bundle urls as column in tags file
start, stop = set_start_stop(df)
df = df.iloc[start:stop]
df.to_csv(outfpath, sep='\n', index=False, header=False)

In [ ]:
#@title Run inference (Note: YOLO cannot parse images from URL, so images are temporarily downloaded) 
# Note: Takes 7-10 min per 5k imgs, aria2 downloads 16imgs at a time
%cd $imgs_dir
!aria2c -x 16 -s 1 -i $tags_file

# Check how many images downloaded
print("Number of files downloaded to Google Drive: ")
len([1 for x in list(os.scandir('.')) if x.is_file()])-1 # -1 because .txt file contains image filenames

# Move tags file used for downloading images to data/img_info/
%cd $cwd
!mv data/imgs/*.txt data/img_info/

# Make a new list of successfully downloaded image files for running inference
inf_imgs = imgs_dir + tags_file
with open(inf_imgs, 'w', encoding='utf-8') as f:
    # Walk through data/imgs/ to list files
    for dir, dirs, files in os.walk(imgs_dir):
        files = [fn for fn in files]
        for fn in files:
            if 'txt' not in fn:
                out = "data/imgs/" + fn
                f.writelines(out + '\n')

# Inspect textfile of images for inference
df = pd.read_table(inf_imgs, header=None, sep='\r')
print("\nNumber of valid images ready for inference in {}: {}".format(inf_imgs, len(df)))

# Run darknet with flag to not show bounding box coordinates
!./darknet detector test cfg/openimages.data cfg/yolov3-openimages.cfg yolov3-openimages.weights -dont_show -save_labels < {outfpath}

print("\n\n~~~\nInference complete! Post-process inference results in next code blocks before running these steps for remaining batches A-D.\n~~~")

In [ ]:
#@title Post-process detection results for each batch

# Combine individual prediction files for each image to all_predictions.txt
df = combine_predictions(imgs_dir)

# Delete inference text files and images (only needed them for inference)
!rm -r data/imgs/*

# Add class names to numeric image tags
tags = add_class_names('data/results/all_predictions.txt')

# Add EOL media URL's from bundle to image tags df
final_tags = add_eolMediaURLs(tags, bundle)

# Save final tags to file
outpath = set_outpath(tags_file)
final_tags.to_csv(outpath, sep="\t", index=False)

print("\n\n~~~\nPost-processing complete! Run above steps for remaining batches A-D before proceeding to next steps.\n~~~")

## Combine tags for 5k image batches A-D
---
After running steps above for each image batch, combine tag files to one 20k tag dataset.

In [ ]:
#@title Define parameters for converting detected classes to desired image tags
%cd $cwd

# Write header row of output tagging file
# Enter any filename from 4 batches of tagging files
tags_file = "flower_fruit_tags_d" #@param ["flower_fruit_tags_d"] {allow-input: true}
tags_fpath = "data/results/" + tags_file + ".tsv"

# Combine exported model predictions and confidence values for all batches
df = combine_tag_files(tags_fpath)

# Filter for desired classes
# These classes will be kept as is
filter1 = ['Flower', 'Fruit'] #@param
pattern1 = '|'.join(filter1)
# These classes will be converted to 'reproductive structures present' (flowers/fruits)
filter2 = ['Food'] #@param
pattern2 = '|'.join(filter2)

# Set all detections for 'Food' to 'Reproductive structures present'
print("\nNo. tags matching filtered class(es) {}: {}\n".format(pattern2, len(df[df.tag.str.match(pattern2)])))
df.loc[df.tag.str.match(pattern2), 'tag'] = 'Reproductive structures present'

# Remove all detections that aren't 'Flower', 'Fruit', or 'Reproductive structures present'
print("\nNo. tags matching filtered class(es) {}: {}\n".format(filter1, len(df[df.tag.str.match(pattern1)])))
print("\nTags matching filtered class(es): \n", df[df.tag.str.match(pattern1)])

patterns = '|'.join(['Flower', 'Fruit', 'Reproductive structures present'])
print("\nNo. tags not matching filtered classes: \n", len(df.tag[~df.tag.str.match(patterns)]))
print("\nTags not matching filtered classes: \n", df[~df.tag.str.match(patterns)])
df.loc[~df.tag.str.match(patterns), 'tag'] = 'None'

# Write results to tsv
outpath = 'data/results/' + tags_file.rsplit('_', 1)[0] + '_final.tsv'
df.to_csv(outpath, sep='\t', index=False)
print("\n\nFinal tagging file {}: \n{}".format(outpath, df.head()))

## Display tagging results on images
---

In [ ]:
#@title Adjust start index and display up to 50 images with their tags
# Suppress warning about too many figures opened
plt.rcParams.update({'figure.max_open_warning': 0})

# Set number of seconds to timeout if image url taking too long to open
import socket
socket.setdefaulttimeout(10)

# Adjust start index using slider
start = 0 #@param {type:"slider", min:0, max:5000, step:50}
stop = min((start+50), len(df))

# Loop through images
for i, row in df.iloc[start:stop].iterrows():
    try:
        # Read in image 
        url = df['eolMediaURL'][i]
        img = url_to_image(url)

        # Fetch image tag
        tag = df['tag'][i]
  
        # Display progress message after each image is loaded
        print('Successfully loaded {} of {} images'.format(i+1, (stop-start)))

        # Plot image with tag
        _, ax = plt.subplots(figsize=(10, 10))
        ax.imshow(img)
        plt.axis('off')
        plt.title('{}) Tag: {}'.format(i+1, tag))

    except:
        print('Check if URL from {} is valid'.format(url))